#**Fraud Detection using regression model**
####**Project overview**
The Tunisian company of electricity and gaz (STEG) is a non-administrative company, it is responsible for delivering electricity and gaz across Tunisia.
The company sufferred tremendous losses in order of 200 million tunisian dinars due to fraudelant manipulations of meters by consumers.

in this project we will be using data gathered by the company (Clients billing/consumption history) to detect and recognize clients involved in fraudelent activities. 

####**Project structure** 
before start writing any code, first we need to understand the problem in details and set a some milestones to achieve in this project.

- Exploratory Data Analysis
  - downloading the data and converting it into a readable dataset
  - understanding the data 
  - Data preparation and cleaning
  - Deciding the features to build the model

- Model creation
  - Decide which type of Model fit to solve the problem in hand
  - Create the model based on input and output shape and the depth of model network
  - Create the model hyperparamater
  - create the training job 

- Visualizing and tuning
  - continue visualizing and tuning the hyperparamaters until we reach a optimized state
  - Model validation and testing

In [1]:
# For Python 2 / 3 compatability
from __future__ import print_function

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from matplotlib import pyplot as plt
import seaborn as sns

# The following lines adjust the granularity of reporting. 
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

print("Imported modules.")

Imported modules.


#Load the dataset
we will be using the dataset provided to us by the ML academy via google drive. The following code cell download the data from google drive, loads it and then creates the following two pandas DataFrames:

*   train_df, which contains the training set

*   test_df, which contains the test set

In [11]:
#Downloading the train.zip file from the provided link https://drive.google.com/file/d/1DxxZ166bZ8MocyM210DODckm926xm4HJ/view?usp=sharing
!gdown --id '1DxxZ166bZ8MocyM210DODckm926xm4HJ'

#Downloading the test.zip file from the provided link https://drive.google.com/file/d/1Unhg1OagxzEmaiCvgqLbRumIoDjx4AcV/view?usp=sharing
!gdown --id '1Unhg1OagxzEmaiCvgqLbRumIoDjx4AcV'

Downloading...
From: https://drive.google.com/uc?id=1DxxZ166bZ8MocyM210DODckm926xm4HJ
To: /content/train.zip
100% 64.2M/64.2M [00:00<00:00, 83.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Unhg1OagxzEmaiCvgqLbRumIoDjx4AcV
To: /content/test.zip
100% 27.7M/27.7M [00:00<00:00, 163MB/s]


In [17]:
from zipfile import ZipFile
files = ['train.zip', 'test.zip']
export_path = "dataset/"
for f in files:
  zf = ZipFile(f, 'r')
  zf.extractall(export_path)
  zf.close()

In [18]:
client_train_df = pd.read_csv("dataset/client_train.csv")
invoice_train_df = pd.read_csv("dataset/invoice_train.csv")
client_test_df = pd.read_csv("dataset/client_test.csv")
invoice_test_df = pd.read_csv("dataset/invoice_test.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
client_train_df.head()

,disrict,client_id,client_catg,region,creation_date,target
0,60,train_Client_0,11,101,31/12/1994,0.0
1,69,train_Client_1,11,107,29/05/2002,0.0
2,62,train_Client_10,11,301,13/03/1986,0.0
3,69,train_Client_100,11,105,11/07/1996,0.0
4,62,train_Client_1000,11,303,14/10/2014,0.0


In [25]:
client_train_df.tail()

,disrict,client_id,client_catg,region,creation_date,target
135488,62,train_Client_99995,11,304,26/07/2004,0.0
135489,63,train_Client_99996,11,311,25/10/2012,0.0
135490,63,train_Client_99997,11,311,22/11/2011,0.0
135491,60,train_Client_99998,11,101,22/12/1993,0.0
135492,60,train_Client_99999,11,101,18/02/1986,0.0


In [27]:
invoice_train_df.head()

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,train_Client_0,2014-03-24,11,1335667,0,203,8,1,82,0,0,0,14302,14384,4,ELEC
1,train_Client_0,2013-03-29,11,1335667,0,203,6,1,1200,184,0,0,12294,13678,4,ELEC
2,train_Client_0,2015-03-23,11,1335667,0,203,8,1,123,0,0,0,14624,14747,4,ELEC
3,train_Client_0,2015-07-13,11,1335667,0,207,8,1,102,0,0,0,14747,14849,4,ELEC
4,train_Client_0,2016-11-17,11,1335667,0,207,9,1,572,0,0,0,15066,15638,12,ELEC


In [29]:
invoice_train_df.tail()

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
4476744,train_Client_99998,2005-08-19,10,1253571,0,202,9,1,400,135,0,0,3197,3732,8,ELEC
4476745,train_Client_99998,2005-12-19,10,1253571,0,202,6,1,200,6,0,0,3732,3938,4,ELEC
4476746,train_Client_99999,1996-09-25,11,560948,0,203,6,1,259,0,0,0,13884,14143,4,ELEC
4476747,train_Client_99999,1996-05-28,11,560948,0,203,6,1,603,0,0,0,13281,13884,4,ELEC
4476748,train_Client_99999,1996-01-25,11,560948,0,203,6,1,516,0,0,0,12765,13281,4,ELEC
